In [11]:
!pip install --upgrade tables
!pip install eli5

     |████████████████████████████████| 4.3MB 2.8MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4


In [0]:
import pandas as pd
import numpy as np

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

##WCZYTANIE DANYCH

In [3]:
cd "/content/drive/My Drive/Colab Notebooks/data_workshop_matrix/Matrix II"

/content/drive/My Drive/Colab Notebooks/data_workshop_matrix/Matrix II


In [4]:
ls

 data/                   'Matrix II Day 3.ipynb'
'Matrix II Day 2.ipynb'  'Matrix II meta.ipynb'


In [0]:
df = pd.read_hdf('data/car.h5')

In [6]:
df.shape

(106494, 155)

In [7]:
df.columns.values

array(['breadcrumb', 'created_at', 'price_currency', 'price_details',
       'price_value', 'seller_address', 'seller_name', 'seller_type',
       'feature_czujniki-parkowania-przednie',
       'feature_poduszka-powietrzna-chroniąca-kolana',
       'feature_kurtyny-powietrzne', 'feature_klimatyzacja-dwustrefowa',
       'feature_światła-led', 'feature_czujnik-zmierzchu',
       'feature_elektrycznie-ustawiane-lusterka',
       'feature_asr-(kontrola-trakcji)',
       'feature_poduszka-powietrzna-kierowcy', 'feature_cd',
       'feature_elektryczne-szyby-przednie',
       'feature_poduszka-powietrzna-pasażera',
       'feature_system-start-stop', 'feature_światła-do-jazdy-dziennej',
       'feature_komputer-pokładowy', 'feature_elektryczne-szyby-tylne',
       'feature_klimatyzacja-manualna', 'feature_tapicerka-welurowa',
       'feature_czujnik-deszczu', 'feature_światła-przeciwmgielne',
       'feature_ogrzewanie-postojowe', 'feature_radio-niefabryczne',
       'feature_regulowane-zaw

## Dummy Model

In [8]:
df.select_dtypes(np.number).columns

Index(['price_value', 'car_id'], dtype='object')

In [0]:
X = df[['car_id']].values
y = df['price_value'].values

In [14]:
model = DummyRegressor()
model.fit(X,y)
y_pred = model.predict(X)

mae(y,y_pred)

39465.934630440985

In [15]:
[x for x in df.columns if 'price' in x]

['price_currency', 'price_details', 'price_value']

In [16]:
df['price_currency'].value_counts()

PLN    106290
EUR       204
Name: price_currency, dtype: int64

In [0]:
df = df[df['price_currency'] != 'EUR']

In [18]:
df['price_currency'].value_counts()

PLN    106290
Name: price_currency, dtype: int64

## FEATURE ENGINEERING

In [0]:
SUFFIX_CAT = '__cat'
for feat in df.columns:
  if isinstance(df[feat][0], list): continue
  factorized_values = df[feat].factorize()[0]
  if SUFFIX_CAT in feat:
    df[feat] = factorized_values
  else:
    df[feat+SUFFIX_CAT] = factorized_values

In [0]:
cat_feats = [x for x in df.columns if SUFFIX_CAT in x]
cat_feats = [x for x in cat_feats if 'price' not in x]

In [31]:
len(cat_feats)

151

In [0]:
X = df[cat_feats].values
y = df['price_value'].values

In [33]:
model = DummyRegressor()
model.fit(X,y)
y_pred = model.predict(X)

mae(y,y_pred)

39456.885927541574

In [34]:
model = DecisionTreeRegressor(max_depth=5)
scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_error')
np.mean(scores)

-19566.588937368328

In [36]:
m = DecisionTreeRegressor(max_depth=5)
m.fit(X,y)

imp = PermutationImportance(m, random_state=8888).fit(X,y)
eli5.show_weights(imp, feature_names = cat_feats)

Weight,Feature
0.2557 ± 0.0056,param_napęd__cat
0.2017 ± 0.0038,param_faktura-vat__cat
0.1940 ± 0.0043,param_stan__cat
0.1451 ± 0.0076,param_rok-produkcji__cat
0.0638 ± 0.0042,param_moc__cat
0.0415 ± 0.0015,param_skrzynia-biegów__cat
0.0414 ± 0.0005,feature_kamera-cofania__cat
0.0286 ± 0.0033,param_marka-pojazdu__cat
0.0197 ± 0.0016,param_pojemność-skokowa__cat
0.0164 ± 0.0005,feature_bluetooth__cat
